In [187]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

URL_EU_TRANSACTION_LOG = "https://climate.ec.europa.eu/document/download/ebb2c20e-8737-4a73-b6ba-a4b7e78ecc01_en?filename=verified_emissions_2023_en_1.xlsx"
URL_EU_ETS_OPERATORS = "https://climate.ec.europa.eu/document/download/ab2c1214-decb-40bc-bb0d-d37f080bdebd_en?filename=policy_ets_registry_operators_ets_en.xlsx"
URL_CO2E_PRICE_DEVELOPMENT = "https://www.umweltbundesamt.de/sites/default/files/medien/384/bilder/dateien/2_abb_preisentwick-emissionsber-eua_2023-11-23.xlsx"
URL_GLOBAL_GHG_EMSISSIONS = "https://edgar.jrc.ec.europa.eu/booklet/EDGARv8.0_FT2022_GHG_booklet_2023.xlsx"

# for local development
# URL_EU_TRANSACTION_LOG = "../data/verified_emissions_2023_en_1.xlsx"
# URL_EU_ETS_OPERATORS = "../data/policy_ets_registry_operators_ets_en.xlsx"
# URL_CO2E_PRICE_DEVELOPMENT = "../data/2_abb_preisentwick-emissionsber-eua_2023-11-23.xlsx"
# URL_GLOBAL_GHG_EMSISSIONS = "../data/EDGARv8.0_FT2022_GHG_booklet_2023.xlsx"

In [188]:
# classes that'll contain the cleaned datasets
class EuropeanUnionTransactionLog:
    raw: pd.DataFrame
    cleaned: pd.DataFrame

class EUETSOperators:
    file: pd.ExcelFile
    raw: pd.DataFrame
    cleaned: pd.DataFrame

class CO2ePriceDevelopment:
    file: pd.ExcelFile
    raw: pd.DataFrame
    cleaned: pd.DataFrame

class GlobalGHGEmissions:
    file: pd.ExcelFile
    raw: pd.DataFrame
    cleaned: pd.DataFrame

eu_transaction_log = EuropeanUnionTransactionLog()
eu_ets_operators = EUETSOperators()
co2e_price_development = CO2ePriceDevelopment()
global_ghg_emissions = GlobalGHGEmissions()

In [189]:
# parse all separate datasets
eu_transaction_log.file = pd.ExcelFile(URL_EU_TRANSACTION_LOG)
eu_ets_operators.file= pd.ExcelFile(URL_EU_ETS_OPERATORS)
co2e_price_development.file = pd.ExcelFile(URL_CO2E_PRICE_DEVELOPMENT)
global_ghg_emissions.file = pd.ExcelFile(URL_GLOBAL_GHG_EMSISSIONS)

In [190]:
# clean eu transaction log dataset
eu_transaction_log_data = pd.read_excel(eu_transaction_log.file, header=21, sheet_name="data")
eu_transaction_log_activity_codes = pd.read_excel(eu_transaction_log.file, sheet_name="activity codes")

# merge with activity code description table
eu_transaction_log.raw = pd.merge(
    left=eu_transaction_log_data,
    right=eu_transaction_log_activity_codes,
    left_on="MAIN_ACTIVITY_TYPE_CODE",
    right_on="code",
)

eu_transaction_log.raw = eu_transaction_log.raw.drop(columns=["code"])
eu_transaction_log.raw = eu_transaction_log.raw.rename(
    columns={
        "value": "MAIN_ACTIVITY",
        "ALLOCATION2008": "ALLOCATION_2008",
    }
)

eu_transaction_log.cleaned = pd.DataFrame(
    columns=[
        "REGISTRY_CODE",
        "IDENTIFIER_IN_REG",
        "INSTALLATION_NAME",
        "INSTALLATION_IDENTIFIER",
        "ALLOCATION",
        "VERIFIED_EMISSIONS",
        "MAIN_ACTIVITY_TYPE_CODE",
        "MAIN_ACTIVITY",
        "YEAR",
    ]
)

for i in range(2008, 2024):
    df_i = eu_transaction_log.raw[
        [
            "REGISTRY_CODE",
            "IDENTIFIER_IN_REG",
            "INSTALLATION_NAME",
            "INSTALLATION_IDENTIFIER",
            f"ALLOCATION_{i}",
            f"VERIFIED_EMISSIONS_{i}",
            "MAIN_ACTIVITY_TYPE_CODE",
            "MAIN_ACTIVITY",
        ]
    ]

    df_i = df_i.rename(columns=lambda x: x.removesuffix(f"_{i}"))
    df_i = df_i.assign(YEAR=i)
    df_i["EXCLUDED"] = df_i["VERIFIED_EMISSIONS"].apply(lambda x: True if x == "Excluded" else False)
    df_i["VERIFIED_EMISSIONS"] = df_i["VERIFIED_EMISSIONS"].apply(lambda x: -1 if x == "Excluded" else x)

    # sum up verified emissions / allocation and ch verified emissions / allocation
    if f"CH_VERIFIED_EMISSIONS_{i}" in eu_transaction_log.raw.columns:
        eu_transaction_log.raw[f"CH_VERIFIED_EMISSIONS_{i}"] = eu_transaction_log.raw[f"CH_VERIFIED_EMISSIONS_{i}"].apply(lambda x: 0 if x == -1 or x == "Excluded" else x)
        df_i["VERIFIED_EMISSIONS"] = df_i["VERIFIED_EMISSIONS"] + eu_transaction_log.raw[f"CH_VERIFIED_EMISSIONS_{i}"]

    if f"CH_ALLOCATION_{i}" in eu_transaction_log.raw.columns:
        eu_transaction_log.raw[f"CH_ALLOCATION_{i}"] = eu_transaction_log.raw[f"CH_ALLOCATION_{i}"].apply(lambda x: 0 if x == -1 or x == "Excluded" else x)
        df_i["ALLOCATION"] = df_i["ALLOCATION"] + eu_transaction_log.raw[f"CH_ALLOCATION_{i}"]

    eu_transaction_log.cleaned = pd.concat([eu_transaction_log.cleaned, df_i])


eu_transaction_log.cleaned.head(n=100)

,REGISTRY_CODE,IDENTIFIER_IN_REG,INSTALLATION_NAME,INSTALLATION_IDENTIFIER,ALLOCATION,VERIFIED_EMISSIONS,MAIN_ACTIVITY_TYPE_CODE,MAIN_ACTIVITY,YEAR,EXCLUDED
0,AT,Michael Strasser GmbH & Co. KG,13030,201505,-1,-1,10,Aircraft operator activities,2008,False
1,AT,Georgetown Management LLC,15451,201836,-1,-1,10,Aircraft operator activities,2008,False
2,AT,PSC Ukraine International Airlines,19210,201564,-1,-1,10,Aircraft operator activities,2008,False
3,AT,Glock Gesellschaft m.b.H.,194,200108,-1,-1,10,Aircraft operator activities,2008,False
4,AT,Air Charter Limited,209,200180,-1,-1,10,Aircraft operator activities,2008,False
...,...,...,...,...,...,...,...,...,...,...
95,AT,Frantschach St. Gertraud,Frantschach St. Gertraud,160,50196,32331,36,Production of paper or cardboard,2008,False
96,AT,Fritz Egger St. Johann Tirol,Fritz Egger St. Johann Tirol,188,25512,24160,20,Combustion of fuels,2008,False
97,AT,Fritz Egger Unterradlberg,Fritz Egger Unterradlberg,189,13785,12834,20,Combustion of fuels,2008,False
98,AT,Fritz Egger Wörgl,Fritz Egger Wörgl,190,20024,16414,20,Combustion of fuels,2008,False


In [191]:
# clean eu ets operators dataset
eu_ets_operators.raw = pd.read_excel(eu_ets_operators.file)
eu_ets_operators.cleaned = eu_ets_operators.raw
eu_ets_operators.cleaned.head()

,Account Holder Name,Company Registration Nr of Account Holder,LEI,MS Registry,Installation ID,Installation Name,Activity Type,Permit ID,PERMIT_REVOCATION_DATE,Permit Expiry/Revocation Date,Contact Country,Contact City,Contact PCode,Contact Address L1,Contact Address L2
0,Calmit GmbH,FN 87132 k,NaN,AT,1,Calmit Bad Ischl,"Production of lime, or calcination of dolomite...",IKA119,NaN,ACTIVE,AT,Bad Ischl,4820,Linzer Straße 8,NaN
1,Breitenfeld Edelstahl AG,FN 74471 t,5299-00RVRNBYTWP2VG-82,AT,2,Breitenfelder Edelstahl Mitterdorf,Production of pig iron or steel,IES069,NaN,ACTIVE,AT,Mitterdorf,8662,Breitenfeldstrasse 22,NaN
2,Ziegelwerk Danreiter GmbH & Co KG,FN 18162 i,NaN,AT,3,Ziegelwerk Danreiter Ried im Innkreis,Manufacture of ceramics,IZI155,NaN,ACTIVE,AT,Tumeltsham,4911,Ottenbach 14,NaN
3,Wienerberger Österreich GmbH,FN 94684 t,5299-00LTBZABEGV8SX-24,AT,4,Wienerberger Blindenmarkt,Installations for the manufacture of ceramic p...,IZI146-1,13/03/08,REVOKED,AT,Wien,1100,Wienerbergerplatz 1,NaN
4,FunderMax GmbH,FN 90081 y,5299-00PB0IFW0P0V8M-54,AT,5,FunderMax Wr. Neudorf,Combustion of fuels,ICH113,NaN,ACTIVE,AT,St. Veit/Glan,9300,Klagenfurter Str. 87-89,NaN


In [206]:
# clean co2e price development dataset
co2e_price_development.raw = pd.read_excel(
    co2e_price_development.file, 
    sheet_name="Daten", 
    header=9, 
    usecols="B,C", 
    names=["date", "price"]
)

co2e_price_development.cleaned = co2e_price_development.raw.dropna()
co2e_price_development.cleaned.head()

,date,price
0,2008-01-03,23.54
1,2008-01-04,23.55
2,2008-01-07,23.66
3,2008-01-08,23.70
4,2008-01-09,23.42


In [193]:
# clean global ghg emissions dataset
global_ghg_emissions.raw = pd.read_excel(global_ghg_emissions.file, sheet_name="GHG_totals_by_country")
global_ghg_emissions.cleaned = global_ghg_emissions.raw.dropna()
global_ghg_emissions.cleaned.tail()

,EDGAR Country Code,Country,1970,1971,1972,1973,1974,1975,1976,1977,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
207,ZAF,South Africa,243.936906,243.544531,250.939841,270.120645,279.293423,297.591817,313.713625,316.015052,...,601.888491,615.452486,590.086767,590.391418,602.773396,605.788699,611.136564,551.464312,548.475888,534.531652
208,ZMB,Zambia,12.784942,13.054113,13.658804,14.136910,14.379248,14.693533,15.145700,14.920682,...,24.162291,24.830244,24.986546,25.575795,27.241617,28.423766,28.645330,28.953793,29.684857,30.330585
209,ZWE,Zimbabwe,22.245786,22.859421,22.516027,24.140295,24.239679,23.566684,24.709480,23.932646,...,32.927064,30.691499,31.610375,29.676863,29.174904,31.273375,30.207339,28.341567,29.525357,30.190141
211,EU27,EU27,4611.325766,4637.851097,4788.933368,5005.025057,4963.517082,4878.009116,5145.629069,5133.688874,...,4019.150279,3869.441688,3922.022084,3926.561741,3952.329493,3870.920359,3713.097654,3427.439824,3617.735706,3587.796150
213,GLOBAL TOTAL,GLOBAL TOTAL,24497.544419,24582.920914,25521.454040,26733.859337,26742.384120,26798.454097,27950.655931,28635.109842,...,49875.675088,50242.997508,50134.383764,50343.044450,51195.419111,52398.143260,52557.335275,50632.309454,53056.607721,53786.038909


In [207]:
# save all cleaned datasets to sqlite
conn = sqlite3.connect("../data/data.sqlite")

eu_transaction_log.cleaned.to_sql("eu_transaction_log", conn, if_exists="replace", index=True)
eu_ets_operators.cleaned.to_sql("eu_ets_operators", conn, if_exists="replace", index=True)
co2e_price_development.cleaned.to_sql("co2e_price_development", conn, if_exists="replace", index=True)
global_ghg_emissions.cleaned.to_sql("global_ghg_emissions", conn, if_exists="replace", index=True)

212